## Imports



In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import gc
import copy

## Data loading

In [ ]:
# Loading of the datasets of 10 nodes
number_nodes = 10
train_loader_10 = np.loadtxt("tsp-data/tsp10_train_concorde.txt", usecols=np.concatenate((np.arange(0,number_nodes*2),np.arange(number_nodes*2+1,number_nodes*3+2))))
val_loader_10 = np.loadtxt("tsp-data/tsp10_val_concorde.txt", usecols=np.concatenate((np.arange(0,number_nodes*2),np.arange(number_nodes*2+1,number_nodes*3+2))))
test_loader_10 = np.loadtxt("tsp-data/tsp10_test_concorde.txt", usecols=np.concatenate((np.arange(0,number_nodes*2),np.arange(number_nodes*2+1,number_nodes*3+2))))

In [ ]:
# Splitting between point coordinates and path
train_input_10 = train_loader_10[:,:number_nodes*2]
train_input_10 = train_input_10.reshape(train_input_10.shape[0], number_nodes, 2)
train_input_10 = torch.Tensor(train_input_10)
train_path_10 = train_loader_10[:,number_nodes*2:]

val_input_10 = val_loader_10[:,:number_nodes*2]
val_input_10 = val_input_10.reshape(val_input_10.shape[0], number_nodes, 2)
val_input_10 = torch.Tensor(val_input_10)
val_path_10 = val_loader_10[:,number_nodes*2:]

test_input_10 = test_loader_10[:,:number_nodes*2]
test_input_10 = test_input_10.reshape(test_input_10.shape[0], number_nodes, 2)
test_input_10 = torch.Tensor(test_input_10)
test_path_10 = test_loader_10[:,number_nodes*2:]

In [ ]:
#Loading of the datasets of 20 nodes
number_nodes = 20
train_loader_20 = np.loadtxt("tsp-data/tsp20_train_concorde.txt", usecols=np.concatenate((np.arange(0,number_nodes*2),np.arange(number_nodes*2+1,number_nodes*3+2))))
val_loader_20 = np.loadtxt("tsp-data/tsp20_val_concorde.txt", usecols=np.concatenate((np.arange(0,number_nodes*2),np.arange(number_nodes*2+1,number_nodes*3+2))))
test_loader_20 = np.loadtxt("tsp-data/tsp20_test_concorde.txt", usecols=np.concatenate((np.arange(0,number_nodes*2),np.arange(number_nodes*2+1,number_nodes*3+2))))

In [ ]:
# Splitting between point coordinates and path
train_input_20 = train_loader_20[:,:number_nodes*2]
train_input_20 = train_input_20.reshape(train_input_20.shape[0], number_nodes, 2)
train_input_20 = torch.Tensor(train_input_20)
train_path_20 = train_loader_20[:,number_nodes*2:]

val_input_20 = val_loader_20[:,:number_nodes*2]
val_input_20 = val_input_20.reshape(val_input_20.shape[0], number_nodes, 2)
val_input_20 = torch.Tensor(val_input_20)
val_path_20 = val_loader_20[:,number_nodes*2:]

test_input_20 = test_loader_20[:,:number_nodes*2]
test_input_20 = test_input_20.reshape(test_input_20.shape[0], number_nodes, 2)
test_input_20 = torch.Tensor(test_input_20)
test_path_20 = test_loader_20[:,number_nodes*2:]

## Definition of the model

In [ ]:
class Embedding(nn.Module):
  def __init__(self, h, k, is_gpu):
    super(Embedding, self).__init__()
    #Parameter
    self.k = k
    #Linear layer for node embedding
    self.linear_node = nn.Linear(2, h, bias = True)
    #Linear layers for edge embedding
    self.linear_distance = nn.Linear(1, h//2, bias = True)
    self.linear_delta = nn.Linear(1, h//2, bias = False)
    self.is_gpu = is_gpu
  
  def forward(self, nodes):
    #Computation of distance matrix
    distances = (((nodes[:,None,:,:] - nodes[:,:,None,:])**2).sum(axis=3)).sqrt()
    if self.is_gpu :     
      distances = distances.cuda()

    #Computation of delta matrix
    delta = torch.zeros(distances.shape)
    if self.is_gpu :
      delta = delta.cuda()
    self_connection = np.arange(distances.shape[1])
    indices = torch.argsort(distances, axis = 2)[:,:,0:self.k+1]
    delta = delta.scatter(2, indices, 1)
    delta[:,self_connection,self_connection] += 1

    #Node embedding
    emb_nodes = self.linear_node(nodes)

    #Edge embedding
    emb_distances = self.linear_distance(distances[:,:,:,None])
    emb_delta = self.linear_delta(delta[:,:,:,None])
    emb_edges = torch.cat((emb_distances, emb_delta), 3)

    return emb_nodes, emb_edges

In [ ]:
class Graph_conv_layer(nn.Module):
  def __init__(self, h, epsilon, is_gpu):
    super(Graph_conv_layer, self).__init__()
    self.is_gpu = is_gpu
    #Parameter
    self.epsilon = epsilon
    #Linear layers
    self.linear_W = []
    for i in range(5):
      l = nn.Linear(h, h, bias = False)
      if self.is_gpu :
        l.cuda()
      self.linear_W.append(l)
    self.linear_W = nn.ModuleList(self.linear_W)
    #Non-linearities
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
    #Batch normalizations
    self.batch_norm_nodes = nn.BatchNorm1d(h)
    if self.is_gpu :
      self.batch_norm_nodes.cuda()
    self.batch_norm_edges = nn.BatchNorm2d(h)
    if self.is_gpu :
      self.batch_norm_edges.cuda()

  def forward(self, emb_nodes, emb_edges):
    #Computation of (emb_nodes)^(l+1) which is called new_emb_nodes in the following
    #Linear layers
    linear_nodes_1 = self.linear_W[0](emb_nodes)
    linear_nodes_2 = self.linear_W[1](emb_nodes)
    #Computation of eta
    sigmoid_edges = self.sigmoid(emb_edges)
    eta = sigmoid_edges / (sigmoid_edges.sum(dim=2)[:,:,None,:]+self.epsilon)
    #Sum of the two terms
    sum_nodes = linear_nodes_1 + (eta * linear_nodes_2[:,:,None,:]).sum(dim=2)
    #Batch normalization
    sum_nodes_bn = self.batch_norm_nodes(sum_nodes.permute((0,2,1))).permute((0,2,1))
    #Computation of new_emb_nodes
    new_emb_nodes = emb_nodes + self.relu(sum_nodes_bn)

    #Computation of (emb_edges)^(l+1) which is called new_emb_edges in the following
    #Linear layers
    linear_edges = self.linear_W[2](emb_edges)
    linear_nodes_1 = self.linear_W[3](emb_nodes)
    linear_nodes_2 = self.linear_W[4](emb_nodes)
    #Sum of the three terms
    sum_edges = linear_edges + linear_nodes_1[:,None,:,:] + linear_nodes_2[:,:,None,:]
    #Batch normalization
    sum_edges_bn = self.batch_norm_edges(sum_edges.permute((0,3,1,2))).permute((0,2,3,1))
    #Computation of new_emb_edges
    new_emb_edges = emb_edges + self.relu(sum_edges_bn)

    return new_emb_nodes, new_emb_edges

In [ ]:
class MLP(nn.Module):
  def __init__(self, dim_layers, is_gpu):
    super(MLP, self).__init__()
    self.is_gpu = is_gpu
    #Parameter
    self.nb_layers = len(dim_layers)
    #Linear layers
    self.linears = []
    for i in range(self.nb_layers):
      l = nn.Linear(dim_layers[i][0], dim_layers[i][1], bias = True)
      if self.is_gpu :
        l.cuda()
      self.linears.append(l)
    self.linears = nn.ModuleList(self.linears)
    #Non-linearity
    self.relu = nn.ReLU()
  
  def forward(self, emb_edges):
    #Initialization
    res = emb_edges

    #Linear layer + non-linearity for each layer except the last one
    for i in range(self.nb_layers-1):
      res = self.linears[i](res)
      res = self.relu(res)
    
    #Last linear layer (no non-linearity)
    res = self.linears[self.nb_layers-1](res)
    
    return(res)

In [ ]:
class Net(nn.Module):
  def __init__(self, h, k, dim_hidden_mlp, nb_graph_layers, epsilon, is_gpu):
    super(Net, self).__init__()
    #Parameter
    self.nb_graph_layers = nb_graph_layers
    #Computation of dim_layers_mlp
    dim_layers_mlp = [[h, dim_hidden_mlp[0]]]
    for i in range(len(dim_hidden_mlp)-1):
      dim_layers_mlp.append([dim_hidden_mlp[i], dim_hidden_mlp[i+1]])
    dim_layers_mlp.append([dim_hidden_mlp[len(dim_hidden_mlp)-1], 2])
    #Input layer
    self.embedding = Embedding(h, k, is_gpu)
    #Graph convolution layers
    self.graph_layers = nn.ModuleList()
    for i in range(self.nb_graph_layers):
      self.graph_layers.append(Graph_conv_layer(h,epsilon, is_gpu))
    #MLP
    self.mlp = MLP(dim_layers_mlp, is_gpu)
  

  def forward(self, input):
    #Input layer
    emb_nodes, emb_edges = self.embedding(input)

    #Graph convolution layers
    for i in range(self.nb_graph_layers):
      emb_nodes, emb_edges = self.graph_layers[i](emb_nodes, emb_edges)

    #MLP
    output = self.mlp(emb_edges)
    
    return(output)

## Useful functions

In [ ]:
def adjacency(path):
  # Returns the adjacency matrix of a path from the dataset
  path = torch.tensor(path)
  heads = path.int()
  tails = torch.roll(heads,-1)
  adjacency_matrix = np.zeros((len(path),len(path)))
  adjacency_matrix[heads,tails] = 1
  adjacency_matrix = torch.tensor(adjacency_matrix, dtype = torch.int64)
  return adjacency_matrix

In [ ]:
def adjacency_matrix(path):
  # Returns the adjacency matrix of a path which is the output of the network
  return(adjacency(path[:-1]-1))

In [ ]:
def dist(nodes, path, target):
  # Returns the distance of the path for these nodes 
  # The path has not the same format if it is coming from the dataset (target=True) or from the network output (target=False)
  if target:
    adj_matrix = torch.Tensor(np.apply_along_axis(adjacency, 1, path))
  else:
    adj_matrix = torch.Tensor(np.apply_along_axis(adjacency_matrix, 1, path))
  distances = (((nodes[:,None,:,:] - nodes[:,:,None,:])**2).sum(axis=3)).sqrt()
  return (adj_matrix * distances).sum(axis=(1,2))

## Beamsearch

In [ ]:
def beamsearch_batch(probabilities, w_b, random=True):
  # Beamsearch algorithm applied on the probabilities with a beam width of w_b 
  batch,nb_nodes,_ = probabilities.shape
  a = np.ones((batch,w_b))
  index = -1*np.ones((batch,w_b, nb_nodes))
  if random:
    init = np.random.randint(0, nb_nodes, (batch,w_b))
  else:
    init = np.zeros((batch,w_b), dtype=np.int8)
  np.put_along_axis(index, init[:,:,None], 0, 2)
  for i in range(nb_nodes-1):
    prob = a[:,:,None]*np.take_along_axis(probabilities,np.argmax(index==i, axis=2)[:,:,None], axis=1)*(index==-1)
    argsort_prob = np.argsort(prob.reshape((batch,w_b*nb_nodes)), axis=1)
    sort_index = np.take_along_axis(index, (argsort_prob//nb_nodes)[:,:,None], axis=1)[:,-w_b:]
    np.put_along_axis(sort_index, (argsort_prob%nb_nodes)[:,-w_b:,None], i+1, 2)
    index = sort_index
    a = np.sort(prob.reshape((batch,w_b*nb_nodes)), axis=1)[:,-w_b:]
  return np.argsort(index, axis=2)

## Optimality gap

In [ ]:
def optimality_gap_greedy(val_input,val_output, val_path):
  # Returns the optimality gap with the greedy method applied on val_path
  prob = np.exp(val_output.detach().numpy())
  beam_search = beamsearch_batch(prob[:,:,:,1],1,False)
  d1 = dist(val_input, beam_search[:,0,:], True)
  d2 = dist(val_input, val_path, False)
  return((d1/d2-1).mean())

In [ ]:
def optimality_gap_beamsearch(val_input,val_output, val_path):
  # Returns the optimality gap with the beamsearch method applied on val_path
  prob = np.exp(val_output.detach().numpy())
  beam_search = beamsearch_batch(prob[:,:,:,1],1280)
  d1 = dist(val_input, beam_search[:,-1,:], True)
  d2 = dist(val_input, val_path, False)
  return((d1/d2-1).mean())

In [ ]:
def optimality_gap_beamsearch_shortest(val_input,val_output, val_path):
  # Returns the optimality gap with the beamsearch + shortest path method applied on val_path
  prob = np.exp(val_output.detach().numpy())
  beam_search = beamsearch_batch(prob[:,:,:,1],1280)
  beam_search = beam_search.transpose((0,2,1))
  b_shape = beam_search.shape
  beam_search = beam_search.reshape((b_shape[0]*b_shape[1], b_shape[2]))
  def dist_1D(b_search):
    b_search = b_search.reshape((b_shape[0], b_shape[1]))
    return(dist(val_input, b_search, True))
  A = np.apply_along_axis(dist_1D, 0, beam_search)
  d1 = np.min(A, axis=1)
  d2 = dist(val_input, val_path, False)
  return((d1/d2-1).mean())

## Training on one epoch

In [ ]:
def train(network, input, target, optimizer, epoch, n):
  network.train()
  weight = torch.Tensor([ n*n/(n*n-2*n)/2, n*n/(2*n)/2])
  loss_function = nn.NLLLoss(weight.cuda())
  loss_function.cuda()
    
  list_loss = []

  for batch in range(500):
    input_batch = input[ batch*20 : (batch+1)*20 ]
    input_batch = input_batch.cuda()
    
    target_batch = target[ batch*20 : (batch+1)*20 ]
    target_batch = torch.flatten(target_batch, start_dim=0, end_dim=2)
    target_batch = target_batch.cuda()

    optimizer.zero_grad()
    output_batch = network(input_batch)

    output_batch = F.log_softmax(output_batch, dim=3)
    output_batch = torch.flatten(output_batch, start_dim=0, end_dim=2)
    loss = loss_function(output_batch, target_batch)

    loss.backward()
    optimizer.step()
    list_loss.append(loss)
  with open('training_loss.txt','a') as f_training_loss:
    print(torch.tensor(list_loss).mean(), file=f_training_loss)
  gc.collect()  
  torch.cuda.empty_cache()

In [ ]:
def train_multi(network, input_10, target_10, input_20, target_20, optimizer, epoch):
  network.train()
  n=10
  weight_10 = torch.Tensor([ n*n/(n*n-2*n)/2, n*n/(2*n)/2])
  loss_function_10 = nn.NLLLoss(weight_10.cuda())
  loss_function_10.cuda()

  n=20
  weight_20 = torch.Tensor([ n*n/(n*n-2*n)/2, n*n/(2*n)/2])
  loss_function_20 = nn.NLLLoss(weight_20.cuda())
  loss_function_20.cuda()
    
  list_loss = []

  for batch in range(250):
    input_batch = input_10[ batch*20 : (batch+1)*20 ]
    input_batch = input_batch.cuda()
    
    target_batch = target_10[ batch*20 : (batch+1)*20 ]
    target_batch = torch.flatten(target_batch, start_dim=0, end_dim=2)
    target_batch = target_batch.cuda()

    optimizer.zero_grad()
    output_batch = network(input_batch)

    output_batch = F.log_softmax(output_batch, dim=3)
    output_batch = torch.flatten(output_batch, start_dim=0, end_dim=2)
    loss = loss_function_10(output_batch, target_batch)

    loss.backward()
    optimizer.step()
    list_loss.append(loss)

    input_batch = input_20[ batch*20 : (batch+1)*20 ]
    input_batch = input_batch.cuda()
    
    target_batch = target_20[ batch*20 : (batch+1)*20 ]
    target_batch = torch.flatten(target_batch, start_dim=0, end_dim=2)
    target_batch = target_batch.cuda()

    optimizer.zero_grad()
    output_batch = network(input_batch)

    output_batch = F.log_softmax(output_batch, dim=3)
    output_batch = torch.flatten(output_batch, start_dim=0, end_dim=2)
    loss = loss_function_20(output_batch, target_batch)

    loss.backward()
    optimizer.step()
    list_loss.append(loss)
  with open('training_lossMulti.txt','a') as f_training_loss:
    print(torch.tensor(list_loss).mean(), file=f_training_loss)
  gc.collect()  
  torch.cuda.empty_cache()

## Training on one dataset

In [ ]:
#Hyperparameters 
nb_nodes = 10
nb_epoch = 1000
h = 300
k = 20
dim_hidden_mlp = [300, 300]
nb_graph_layers = 30
epsilon = 1e-20
lr = 0.05

# Data
train_input = train_input_10
train_path = train_path_10
val_input = val_input_10
val_path = val_path_10

# Initialization of the network
network = Net(h, k, dim_hidden_mlp, nb_graph_layers, epsilon, True)
network.cuda()

# Initialization of the optimizer and loss function
optimizer = optim.Adam(network.parameters(), lr)
loss = 1
weight = torch.Tensor([ nb_nodes*nb_nodes/(nb_nodes*nb_nodes-2*nb_nodes)/2, nb_nodes*nb_nodes/(2*nb_nodes)/2])
loss_function = nn.NLLLoss(weight)

# Computation of the adjacency matrix of the validation dataset
val_target = torch.zeros((len(val_path), nb_nodes, nb_nodes))
val_path = torch.Tensor(val_path)
for i in range(len(val_path)):
    val_target[i] = adjacency_matrix(val_path[i])
val_target = val_target.type(torch.LongTensor)

# Initialization of the validation model (on the cpu because of RAM problem)
model_val = Net(h, k, dim_hidden_mlp, nb_graph_layers, epsilon, False)

with open('logs/greedy.txt', 'w') as f_greedy:
      with open('logs/beam.txt', 'w') as f_beam:
        with open('logs/shortest.txt', 'w') as f_shortest:
            with open('logs/loss.txt','w') as f_loss:
                for epoch in range(nb_epoch):
                    # Computation of the sub-selection of the training dataset used and of its adjacency matrix
                    sub_selection = np.arange(len(train_path))
                    random.shuffle(sub_selection)
                    sub_selection = sub_selection[:10000]

                    input_batch = torch.Tensor(train_input[sub_selection])
                    path_batch = torch.Tensor(train_path[sub_selection])

                    target_batch = torch.zeros((len(path_batch), nb_nodes, nb_nodes))
                    for i in range(len(path_batch)):
                      target_batch[i] = adjacency_matrix(path_batch[i])
                    target_batch = target_batch.type(torch.LongTensor)

                    # Training
                    train(network, input_batch, target_batch, optimizer, epoch, nb_nodes)

                    # Evaluation every 5 epochs 
                    if epoch%5 == 0:
                        # Initialization 
                        greedy = []
                        beam = []
                        shortest = []
                        losses = []

                        # Sub-selection of a part of the validation dataset
                        sub_selection = np.arange(len(val_path))
                        random.shuffle(sub_selection)
                        sub_selection = sub_selection[:500]
                        val_input_batch = torch.Tensor(val_input[sub_selection])
                        val_path_batch = torch.Tensor(val_path[sub_selection])
                        val_target_batch = torch.Tensor(val_target[sub_selection])
                        val_target_batch = torch.flatten(val_target_batch, start_dim=0, end_dim=2)

                        # Loading of the current model into the cpu ones (because of RAM problem)
                        model_val.load_state_dict(network.state_dict())

                        for batch in range(2):
                          # Evaluation 
                          val_output_cpu = model_val(val_input_batch[100*batch:100*(batch+1)])
                          val_output_cpu = F.log_softmax(val_output_cpu, dim=3)

                          #Computation of the optimalities gap and of the loss
                          greedy.append(optimality_gap_greedy(val_input_batch[100*batch:100*(batch+1)],val_output_cpu, val_path_batch[100*batch:100*(batch+1)]))
                          beam.append(optimality_gap_beamsearch(val_input_batch[100*batch:100*(batch+1)],val_output_cpu, val_path_batch[100*batch:100*(batch+1)]))
                          shortest.append(optimality_gap_beamsearch_shortest(val_input_batch[100*batch:100*(batch+1)],val_output_cpu, val_path_batch[100*batch:100*(batch+1)]))
                          val_output_cpu = torch.flatten(val_output_cpu, start_dim=0, end_dim=2)
                          losses.append(loss_function(val_output_cpu, val_target_batch[100*batch*nb_nodes*nb_nodes:100*(batch+1)*nb_nodes*nb_nodes]))

                          gc.collect()  
                          torch.cuda.empty_cache()

                        # Saving of the optimalities gap and of the loss into files
                        print(torch.tensor(greedy).mean(), file=f_greedy, flush=True)
                        print(torch.tensor(beam).mean(), file=f_beam, flush=True)
                        print(torch.tensor(shortest).mean(), file=f_shortest, flush=True)
                        new_loss = torch.tensor(losses).mean()
                        print(new_loss, file=f_loss, flush=True)

                        # Decreasing of the learning rate if the loss does not sufficiently decrease 
                        if (loss - new_loss) / loss < 0.01:
                          lr /=1.01
                          print('new lr : ',lr)
                          optimizer = optim.Adam(network.parameters(), lr)
                        loss = new_loss

                    # Saving of the model every 20 epochs
                    if (epoch%20 == 0):
                        torch.save(network.state_dict(), 'model.pth')

## Training on two datasets

In [ ]:
#Hyperparameters 
nb_epoch = 1000
h = 300
k = 20
dim_hidden_mlp = [300, 300]
nb_graph_layers = 30
epsilon = 1e-20
lr = 0.001

# Initialization of the network
network = Net(h, k, dim_hidden_mlp, nb_graph_layers, epsilon, True)
network.cuda()

# Initialization of the optimizer and loss function
optimizer = optim.Adam(network.parameters(), lr)
loss = 1
nb_nodes = 10
weight_10 = torch.Tensor([ nb_nodes*nb_nodes/(nb_nodes*nb_nodes-2*nb_nodes)/2, nb_nodes*nb_nodes/(2*nb_nodes)/2])
loss_function_10 = nn.NLLLoss(weight_10)
nb_nodes = 20
weight_20 = torch.Tensor([ nb_nodes*nb_nodes/(nb_nodes*nb_nodes-2*nb_nodes)/2, nb_nodes*nb_nodes/(2*nb_nodes)/2])
loss_function_20 = nn.NLLLoss(weight_20)

# Computation of the adjacency matrix of the two validation datasets
val_target_10 = torch.zeros((len(val_path_10), 10, 10))
val_path_10 = torch.Tensor(val_path_10)
for i in range(len(val_path_10)):
    val_target_10[i] = adjacency_matrix(val_path_10[i])
val_target_10 = val_target_10.type(torch.LongTensor)
val_target_20 = torch.zeros((len(val_path_20), 20, 20))
val_path_20 = torch.Tensor(val_path_20)
for i in range(len(val_path_20)):
    val_target_20[i] = adjacency_matrix(val_path_20[i])
val_target_20 = val_target_20.type(torch.LongTensor)

# Initialization of the validation model (on the cpu because of RAM problem)
model_val = Net(h, k, dim_hidden_mlp, nb_graph_layers, epsilon, False)

with open('logs/greedyMulti.txt', 'w') as f_greedy:
      with open('logs/beamMulti.txt', 'w') as f_beam:
        with open('logs/shortestMulti.txt', 'w') as f_shortest:
            with open('logs/lossMulti.txt','w') as f_loss:
                for epoch in range(nb_epoch):
                    # Computation of the sub-selections of the training datasets used and of their adjacency matrix
                    sub_selection = np.arange(len(train_path_10))
                    random.shuffle(sub_selection)
                    sub_selection = sub_selection[:5000]

                    input_batch_10 = torch.Tensor(train_input_10[sub_selection])
                    path_batch_10 = torch.Tensor(train_path_10[sub_selection])

                    target_batch_10 = torch.zeros((len(path_batch_10), 10, 10))
                    for i in range(len(path_batch_10)):
                      target_batch_10[i] = adjacency_matrix(path_batch_10[i])
                    target_batch_10 = target_batch_10.type(torch.LongTensor)

                    sub_selection = np.arange(len(train_path_20))
                    random.shuffle(sub_selection)
                    sub_selection = sub_selection[:5000]

                    input_batch_20 = torch.Tensor(train_input_20[sub_selection])
                    path_batch_20 = torch.Tensor(train_path_20[sub_selection])

                    target_batch_20 = torch.zeros((len(path_batch_20), 20, 20))
                    for i in range(len(path_batch_20)):
                      target_batch_20[i] = adjacency_matrix(path_batch_20[i])
                    target_batch_20 = target_batch_20.type(torch.LongTensor)

                    # Training
                    train_multi(network, input_batch_10, target_batch_10, input_batch_20, target_batch_20, optimizer, epoch)

                    # Evaluation every 5 epochs 
                    if epoch%5 == 0:
                        # Initialization
                        greedy = []
                        beam = []
                        shortest = []
                        losses = []

                        # Sub-selection of a part of the validation datasets
                        sub_selection = np.arange(len(val_path_10))
                        random.shuffle(sub_selection)
                        sub_selection = sub_selection[:500]
                        val_input_batch_10 = torch.Tensor(val_input_10[sub_selection])
                        val_path_batch_10 = torch.Tensor(val_path_10[sub_selection])
                        val_target_batch_10 = torch.Tensor(val_target_10[sub_selection])
                        val_target_batch_10 = torch.flatten(val_target_batch_10, start_dim=0, end_dim=2)

                        sub_selection = np.arange(len(val_path_10))
                        random.shuffle(sub_selection)
                        sub_selection = sub_selection[:500]
                        val_input_batch_20 = torch.Tensor(val_input_20[sub_selection])
                        val_path_batch_20 = torch.Tensor(val_path_20[sub_selection])
                        val_target_batch_20 = torch.Tensor(val_target_20[sub_selection])
                        val_target_batch_20 = torch.flatten(val_target_batch_20, start_dim=0, end_dim=2)

                        # Loading of the current model into the cpu ones (because of RAM problem)
                        model_val.load_state_dict(network.state_dict())

                        for batch in range(1):
                          # Evaluation on 10 nodes
                          val_output_cpu_10 = model_val(val_input_batch_10[100*batch:100*(batch+1)])
                          val_output_cpu_10 = F.log_softmax(val_output_cpu_10, dim=3)

                          # Computation of the optimalities gap and of the loss on 10 nodes
                          greedy.append(optimality_gap_greedy(val_input_batch_10[100*batch:100*(batch+1)],val_output_cpu_10, val_path_batch_10[100*batch:100*(batch+1)]))
                          beam.append(optimality_gap_beamsearch(val_input_batch_10[100*batch:100*(batch+1)],val_output_cpu_10, val_path_batch_10[100*batch:100*(batch+1)]))
                          shortest.append(optimality_gap_beamsearch_shortest(val_input_batch_10[100*batch:100*(batch+1)],val_output_cpu_10, val_path_batch_10[100*batch:100*(batch+1)]))
                          val_output_cpu_10 = torch.flatten(val_output_cpu_10, start_dim=0, end_dim=2)
                          losses.append(loss_function_10(val_output_cpu_10, val_target_batch_10[100*batch*10*10:100*(batch+1)*10*10]))

                          # Evaluation on 20 nodes
                          val_output_cpu_20 = model_val(val_input_batch_20[100*batch:100*(batch+1)])
                          torch.set_printoptions(profile="full")
                          val_output_cpu_20 = F.log_softmax(val_output_cpu_20, dim=3)

                          # Computation of the optimalities gap and of the loss on 20 nodes
                          greedy.append(optimality_gap_greedy(val_input_batch_20[100*batch:100*(batch+1)],val_output_cpu_20, val_path_batch_20[100*batch:100*(batch+1)]))
                          beam.append(optimality_gap_beamsearch(val_input_batch_20[100*batch:100*(batch+1)],val_output_cpu_20, val_path_batch_20[100*batch:100*(batch+1)]))
                          shortest.append(optimality_gap_beamsearch_shortest(val_input_batch_20[100*batch:100*(batch+1)],val_output_cpu_20, val_path_batch_20[100*batch:100*(batch+1)]))
                          val_output_cpu_20 = torch.flatten(val_output_cpu_20, start_dim=0, end_dim=2)
                          losses.append(loss_function_20(val_output_cpu_20, val_target_batch_20[100*batch*20*20:100*(batch+1)*20*20]))

                          gc.collect()  
                          torch.cuda.empty_cache()

                        # Saving of the optimalities gap and of the loss into files
                        print(torch.tensor(greedy).mean(), file=f_greedy, flush=True)
                        print(torch.tensor(beam).mean(), file=f_beam, flush=True)
                        print(torch.tensor(shortest).mean(), file=f_shortest, flush=True)
                        new_loss = torch.tensor(losses).mean()
                        print(new_loss)
                        print(new_loss, file=f_loss, flush=True)

                        # Decreasing of the learning rate if the loss does not sufficiently decrease
                        if (loss - new_loss) / loss < 0.01:
                          lr /=1.01
                          print('new lr : ',lr)
                          optimizer = optim.Adam(network.parameters(), lr)
                        loss = new_loss
                    
                    # Saving of the model every 20 epochs
                    if (epoch%20 == 0):
                        torch.save(network.state_dict(), 'model_Multi.pth')


## Evaluation

In [ ]:
# 10 nodes
greedy = []
beam = []
shortest = []
for batch in range(len(test_path_10)//100):
    val_output = model_val(test_input_10[100*batch:100*(batch+1)])
    val_output = F.log_softmax(val_output_cpu, dim=3)

    greedy.append(optimality_gap_greedy(test_input_10[100*batch:100*(batch+1)],val_output_cpu, test_path_10[100*batch:100*(batch+1)]))
    beam.append(optimality_gap_beamsearch(test_input_10[100*batch:100*(batch+1)],val_output_cpu, test_path_10[100*batch:100*(batch+1)]))
    shortest.append(optimality_gap_beamsearch_shortest(test_input_10[100*batch:100*(batch+1)],val_output_cpu, test_path_10[100*batch:100*(batch+1)]))

print("greedy ", torch.tensor(greedy).mean())
print("beam ", torch.tensor(beam).mean())
print("shortest ", torch.tensor(shortest).mean())

In [ ]:
# 20 nodes
greedy = []
beam = []
shortest = []
for batch in range(len(test_path_20)//100):
    val_output = model_val(test_input_20[100*batch:100*(batch+1)])
    val_output = F.log_softmax(val_output_cpu, dim=3)

    greedy.append(optimality_gap_greedy(test_input_20[100*batch:100*(batch+1)],val_output_cpu, test_path_20[100*batch:100*(batch+1)]))
    beam.append(optimality_gap_beamsearch(test_input_20[100*batch:100*(batch+1)],val_output_cpu, test_path_20[100*batch:100*(batch+1)]))
    shortest.append(optimality_gap_beamsearch_shortest(test_input_20[100*batch:100*(batch+1)],val_output_cpu, test_path_20[100*batch:100*(batch+1)]))

print("greedy ", torch.tensor(greedy).mean())
print("beam ", torch.tensor(beam).mean())
print("shortest ", torch.tensor(shortest).mean())